In [1]:
import os
import scipy
from keras.preprocessing import image
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense,BatchNormalization
from keras.models import load_model

Now, we will be using ImageDataGenerator.flow_from_directory() for generating the data of each sample in training set and storing it into train_batch for training purpose and into valid_batch for validation.

In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),
              class_mode='categorical' ):
    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',
                                   class_mode=class_mode,target_size=target_size)
"""
target_size - Tuple of integers (height, width), defaults to (256,256). The dimensions to which all images found will be resized.
batch_size - Size of the batches of data (default: 32).
SPE - steps per epochs - Number of train_batches passed to the model such that it train the whole dataset once 
                        (one epoch completed).
VS - validation steps - Number of valid_batches passed to the model after finished training once ( after one epoch).
"""
BS= 32
TS=(24,24)

train_batch= generator('data/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('data/valid',shuffle=True, batch_size=BS,target_size=TS)

print(len(train_batch))
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 1 classes.
39
38 6


The first layer in the model is a 2-dimensional convolutional layer. This layer will have 32 output filters each with a kernel size of 3x3, and we'll use the relu activation function.
On the first layer only, we also specify the input_shape, which is the shape of our data.Here , we have input_shape of (24,24,1).We follow this by adding another convolutional layer with the exact specs as the earlier one, and added one more convolutional layer except for this third Conv2D layer has 64 filters. The choice of 64 here is again arbitrary, but the general choice of having more filters in later layers than in earlier ones is common.

Dense layer is the output layer of the network, and so it has 2 nodes, one for open eyes and one for closed eyes. We'll use the softmax activation function on our output so that the output for each sample is a probability distribution over the outputs of
open eye and  closed eye.

In [4]:
model = Sequential([
    # 32 convolution filters used each of size 3x3
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    
    # 32 convolution filters used each of size 3x3
    Conv2D(32,(3,3),activation='relu'),
    
    # 64 convolution filters used each of size 3x3
    Conv2D(64, (3, 3), activation='relu'),
    
    # dropout - it is a regularization technique that randomly ignores a subset of nodes in a given layer
    # to improve convergence and to reduce overfiting conditions. 
    # A model is said to be underfitting when it's not able to classify the data it was trained on. 
    Dropout(0.25),
    
    #  flattening -  output shape is (batch, 1).
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])
print('model has been created ')

model has been created 


In [5]:
train_batch.class_indices

{'Closed': 0, 'Open': 1}

In [18]:
#Now that the model is built, we compile and train the model using the Adam optimizer,a loss of categorical_cross_entropy
# epochs - An epoch refers to a single pass of the entire dataset to the network during training.

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_batch,validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)
# Avoid retraning the model for avoiding overfitting condition

Epoch 1/15
38/38 [==============================] - 106s 2s/step - loss: 0.0405 - accuracy: 0.9841 - val_loss: 0.0902 - val_accuracy: 0.9635
Epoch 2/15
38/38 [==============================] - 18s 467ms/step - loss: 0.0188 - accuracy: 0.9978 - val_loss: 0.0987 - val_accuracy: 0.9635
Epoch 3/15
38/38 [==============================] - 18s 481ms/step - loss: 0.0103 - accuracy: 0.9966 - val_loss: 0.0671 - val_accuracy: 0.9635
Epoch 4/15
38/38 [==============================] - 17s 446ms/step - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.0358 - val_accuracy: 0.9792
Epoch 5/15
38/38 [==============================] - 28s 755ms/step - loss: 0.0186 - accuracy: 0.9925 - val_loss: 0.1015 - val_accuracy: 0.9688
Epoch 6/15
38/38 [==============================] - 21s 552ms/step - loss: 0.0074 - accuracy: 0.9983 - val_loss: 0.0852 - val_accuracy: 0.9740
Epoch 7/15
38/38 [==============================] - 17s 444ms/step - loss: 0.0052 - accuracy: 0.9978 - val_loss: 0.1411 - val_accuracy: 0.9688
E

In [22]:
model.save('models/cnn.h5', overwrite=True)